# References

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
#sys.path.append('../fastai/old/') #fastai version 0.7
sys.path.append('../fastai/') #fastai version 1

from fastai import *
from fastai.vision import *

import pdb
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
import png
from tqdm import tqdm_notebook as tqdm

In [4]:
torch.cuda.set_device(1)
torch.backends.cudnn.benchmark=True

# Global Variables

In [5]:
DP = Path('../datasets/human-protein-atlas-image-classification/')

TRAIN_PNGS = DP/'train_pngs'

TEST_ONE = DP/'test1_pngs'

SUBMISSIONS = DP/'submissions'
SUBMISSIONS.mkdir(exist_ok=True)

CSVS = DP/'csvs'
CSVS.mkdir(exist_ok=True)

filter_colors = ['blue', 'green', 'red', 'yellow']

IdToCatDict = {0:'Nucleoplasm',
               1:'Nuclear_membrane',
               2:'Nucleoli',
               3:'Nucleoli_fibrillar_center',
               4:'Nuclear_speckles',
               5:'Nuclear_bodies',
               6:'Endoplasmic_reticulum',
               7:'Golgi_apparatus',
               8:'Peroxisomes',
               9:'Endosomes',
               10:'Lysosomes',
               11:'Intermediate_filaments',
               12:'Actin_filaments',
               13:'Focal_adhesion_sites',
               14:'Microtubules',
               15:'Microtubule_ends',
               16:'Cytokinetic_bridge',
               17:'Mitotic_spindle',
               18:'Microtubule_organizing_center',
               19:'Centrosome',
               20:'Lipid_droplets',
               21:'Plasma_membrane',
               22:'Cell_junctions',
               23:'Mitochondria',
               24:'Aggresome',
               25:'Cytosol',
               26:'Cytoplasmic_bodies',
               27:'Rods_&_rings'}

# Data Setup

In [26]:
list(DP.iterdir())

[PosixPath('../datasets/human-protein-atlas-image-classification/train_pngs'),
 PosixPath('../datasets/human-protein-atlas-image-classification/sample_submission.csv'),
 PosixPath('../datasets/human-protein-atlas-image-classification/submissions'),
 PosixPath('../datasets/human-protein-atlas-image-classification/test1_pngs.zip'),
 PosixPath('../datasets/human-protein-atlas-image-classification/train.csv'),
 PosixPath('../datasets/human-protein-atlas-image-classification/.ipynb_checkpoints'),
 PosixPath('../datasets/human-protein-atlas-image-classification/test1_pngs'),
 PosixPath('../datasets/human-protein-atlas-image-classification/csvs'),
 PosixPath('../datasets/human-protein-atlas-image-classification/train_pngs.zip')]

In [27]:
trainIds_df = pd.read_csv(DP/'train.csv')
trainIds_df.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [28]:
df = pd.DataFrame(columns=['fn','clas'])
df = df.append({'fn':'name', 'clas':666}, ignore_index=True)
df

,fn,clas
0,name,666


In [25]:
trainFns_df = pd.DataFrame(columns=['fn','clas'])

for index, row in tqdm(trainIds_df.iterrows(), total=trainIds_df.shape[0]):
    bn = row['Id']
    cats = row['Target']
    for c in filter_colors:
        fn = f'{bn}_{c}.png'
        trainFns_df = trainFns_df.append({'fn': fn, 'clas': cats}, ignore_index=True)

In [29]:
trainFns_df.head()

,fn,clas
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_blue.png,16 0
1,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_green.png,16 0
2,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_red.png,16 0
3,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_yellow.png,16 0
4,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_blue.png,7 1 2 0


In [31]:
trainFns_df.to_csv(CSVS/'trainFnsPngs.csv')